In [29]:
pip install python-rake
pip install summa

Используем следующие алгоритмы: RAKE, TextRank, tf-idf. В качестве текстов мы взяли 4 статьи с киберленинки, в первой строчке написаны ключевые слова (из раздела ключевые слова статьи + те что руками нашли)

In [47]:
import os

texts = []
keywords = []


from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import RAKE
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from summa import keywords as textrank
import numpy as np

# Лемматизация текстов
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if len(t) > 2:
            lemmas.append(
                m.parse(t)[0].normal_form
            )
    return ' '.join(lemmas)


for fn in os.listdir('texts'):
    if fn.endswith('.txt'):
        with open('texts/' + fn, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            keywords.append(list(map(lambda x: normalize_text(x.strip()), lines[0].split(','))))
            texts.append(normalize_text("\n".join(lines[1:])))
            print(f"Число токенов: {len(texts[-1].split())}")

Число токенов: 930
Число токенов: 1189
Число токенов: 1325
Число токенов: 851


тут пара вспомогательных функций для проверки принадлежности ключевой фразы списку морфологических паттернов, а так же функция извлечения ключевых слов при помощи tf-idf

In [48]:
def match_pos(pos, tokens):
    words = tokens.split()
    if len(pos) != len(words):
        return False
    for i in range(len(pos)):
        if pos[i] != m.parse(words[i])[0].tag.POS:
            return False
    return True

morph_patterns = [['NOUN'], ['ADJF'], ['ADJS'], ['VERB'], ['ADJF', 'NOUN'], ['VERB', 'NOUN'],
                  ['NOUN', 'NOUN'], ['NOUN', 'VERB']]
def match_patterns(tokens):
    for patt in morph_patterns:
        if match_pos(patt, tokens):
            return True
    return False

from sklearn.feature_extraction.text import TfidfVectorizer
def get_keywords(texts, treshold=0.1, stop_words=[]):
    tfidfVectorizer = TfidfVectorizer(stop_words=stop, min_df=2, ngram_range=(1,3))
    tfidf_keys = tfidfVectorizer.fit_transform(np.array(list(map(normalize_text, texts)))).todense()
    inv_voc = {value: key for key, value in tfidfVectorizer.vocabulary_.items()}
    extracted_keywords = []
    for i, text in enumerate(texts):
        extracted_keywords.append([])
        for j in range(tfidf_keys.shape[1]):
            if tfidf_keys[i, j] > treshold and inv_voc[j] not in stop_words:
                extracted_keywords[-1].append(inv_voc[j])
    return extracted_keywords

Мы будем немного непревычно считать метрики -- сравнивая извлеченную ключевую фразу с эталонной, мы будем смотреть не точное совпадение, а вхождние одной как подстроки в другую, это отчасти позволит сравнивать их как "одно и то же по смыслу".

## Rake

### Без морфологических шаблонов

In [49]:
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
extracted_keywords = []

for text in texts:
    extracted_keywords_ = rake.run(text, maxWords=3, minFrequency=1)
    extracted_keywords.append(list(map(lambda x: x[0], filter(lambda x: x[1] > 1, extracted_keywords_))))

    
for i, etalon in enumerate(keywords):
    tp_count = 0
    used_words = set()
    for word in extracted_keywords[i]:
        for second_word in etalon:
            if second_word not in used_words and (word in second_word or second_word in word):
                used_words.add(second_word)
                tp_count += 1
                break
                
    pr = tp_count/len(extracted_keywords[i])
    rec = tp_count/len(etalon)
    try:
        f_score = 2*pr*rec/(pr + rec)
    except ZeroDivisionError:
        f_score = 0
    print(f"ТЕКСТ {i}")
    print(f"Точность: {pr}")
    print(f"Полнота: {rec}")
    print(f"F-score: {f_score}")
    print("-------------------------------------------")

ТЕКСТ 0
Точность: 0.375
Полнота: 0.23076923076923078
F-score: 0.2857142857142857
-------------------------------------------
ТЕКСТ 1
Точность: 0.10810810810810811
Полнота: 0.3333333333333333
F-score: 0.163265306122449
-------------------------------------------
ТЕКСТ 2
Точность: 0.03333333333333333
Полнота: 0.125
F-score: 0.052631578947368425
-------------------------------------------
ТЕКСТ 3
Точность: 0.0
Полнота: 0.0
F-score: 0
-------------------------------------------


### С морфологическими шаблонами

In [50]:
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
extracted_keywords = []

for text in texts:
    extracted_keywords_ = rake.run(text, maxWords=3, minFrequency=1)
    extracted_keywords.append(list(filter(match_patterns, map(lambda x: x[0], filter(lambda x: x[1] > 1, extracted_keywords_)))))

    
for i, etalon in enumerate(keywords):
    tp_count = 0
    used_words = set()
    for word in extracted_keywords[i]:
        for second_word in etalon:
            if second_word not in used_words and (word in second_word or second_word in word):
                used_words.add(second_word)
                tp_count += 1
                break
                
    pr = tp_count/len(extracted_keywords[i])
    rec = tp_count/len(etalon)
    try:
        f_score = 2*pr*rec/(pr + rec)
    except ZeroDivisionError:
        f_score = 0
    print(f"ТЕКСТ {i}")
    print(f"Точность: {pr}")
    print(f"Полнота: {rec}")
    print(f"F-score: {f_score}")
    print("-------------------------------------------")

ТЕКСТ 0
Точность: 0.0
Полнота: 0.0
F-score: 0
-------------------------------------------
ТЕКСТ 1
Точность: 0.18181818181818182
Полнота: 0.16666666666666666
F-score: 0.17391304347826086
-------------------------------------------
ТЕКСТ 2
Точность: 0.0
Полнота: 0.0
F-score: 0
-------------------------------------------
ТЕКСТ 3
Точность: 0.0
Полнота: 0.0
F-score: 0
-------------------------------------------


## TextRank

### Без морфологических шаблонов

In [51]:
stop = stopwords.words('russian')
extracted_keywords = []

for text in texts:
    extracted_keywords_ = textrank.keywords(normalize_text(text), language='russian', additional_stopwords=stop, scores=True)
    extracted_keywords.append(list(map(lambda x: x[0], extracted_keywords_)))

    
for i, etalon in enumerate(keywords):
    tp_count = 0
    used_words = set()
    for word in extracted_keywords[i]:
        for second_word in etalon:
            if second_word not in used_words and (word in second_word or second_word in word):
                used_words.add(second_word)
                tp_count += 1
                break
                
    pr = tp_count/len(extracted_keywords[i])
    rec = tp_count/len(etalon)
    try:
        f_score = 2*pr*rec/(pr + rec)
    except ZeroDivisionError:
        f_score = 0
    print(f"ТЕКСТ {i}")
    print(f"Точность: {pr}")
    print(f"Полнота: {rec}")
    print(f"F-score: {f_score}")
    print("-------------------------------------------")

ТЕКСТ 0
Точность: 0.11392405063291139
Полнота: 0.6923076923076923
F-score: 0.1956521739130435
-------------------------------------------
ТЕКСТ 1
Точность: 0.08823529411764706
Полнота: 0.5
F-score: 0.15
-------------------------------------------
ТЕКСТ 2
Точность: 0.030927835051546393
Полнота: 0.375
F-score: 0.05714285714285715
-------------------------------------------
ТЕКСТ 3
Точность: 0.05063291139240506
Полнота: 0.5
F-score: 0.09195402298850573
-------------------------------------------


### С морфологическими шаблонами

In [52]:
stop = stopwords.words('russian')
extracted_keywords = []

for text in texts:
    extracted_keywords_ = textrank.keywords(normalize_text(text), language='russian', additional_stopwords=stop, scores=True)
    extracted_keywords.append(list(filter(match_patterns, map(lambda x: x[0], extracted_keywords_))))

    
for i, etalon in enumerate(keywords):
    tp_count = 0
    used_words = set()
    for word in extracted_keywords[i]:
        for second_word in etalon:
            if second_word not in used_words and (word in second_word or second_word in word):
                used_words.add(second_word)
                tp_count += 1
                break
                
    pr = tp_count/len(extracted_keywords[i])
    rec = tp_count/len(etalon)
    try:
        f_score = 2*pr*rec/(pr + rec)
    except ZeroDivisionError:
        f_score = 0
    print(f"ТЕКСТ {i}")
    print(f"Точность: {pr}")
    print(f"Полнота: {rec}")
    print(f"F-score: {f_score}")
    print("-------------------------------------------")

ТЕКСТ 0
Точность: 0.1590909090909091
Полнота: 0.5384615384615384
F-score: 0.2456140350877193
-------------------------------------------
ТЕКСТ 1
Точность: 0.11764705882352941
Полнота: 0.3333333333333333
F-score: 0.1739130434782609
-------------------------------------------
ТЕКСТ 2
Точность: 0.04918032786885246
Полнота: 0.375
F-score: 0.08695652173913043
-------------------------------------------
ТЕКСТ 3
Точность: 0.07142857142857142
Полнота: 0.5
F-score: 0.125
-------------------------------------------


## TF-IDF

### Без морфологических шаблонов

In [54]:
stop = stopwords.words('russian')
extracted_keywords = []

extracted_keywords = get_keywords(texts, stop_words=stop)

for i, etalon in enumerate(keywords):
    tp_count = 0
    used_words = set()
    for word in extracted_keywords[i]:
        for second_word in etalon:
            if second_word not in used_words and (word in second_word or second_word in word):
                used_words.add(second_word)
                tp_count += 1
                break
                
    pr = tp_count/len(extracted_keywords[i])
    rec = tp_count/len(etalon)
    try:
        f_score = 2*pr*rec/(pr + rec)
    except ZeroDivisionError:
        f_score = 0
    print(f"ТЕКСТ {i}")
    print(f"Точность: {pr}")
    print(f"Полнота: {rec}")
    print(f"F-score: {f_score}")
    print("-------------------------------------------")

ТЕКСТ 0
Точность: 0.7142857142857143
Полнота: 0.38461538461538464
F-score: 0.5
-------------------------------------------
ТЕКСТ 1
Точность: 0.17647058823529413
Полнота: 0.25
F-score: 0.20689655172413793
-------------------------------------------
ТЕКСТ 2
Точность: 0.05263157894736842
Полнота: 0.125
F-score: 0.07407407407407407
-------------------------------------------
ТЕКСТ 3
Точность: 0.14285714285714285
Полнота: 0.375
F-score: 0.20689655172413796
-------------------------------------------


### С морфологическими шаблонами

In [55]:
stop = stopwords.words('russian')
extracted_keywords = []

extracted_keywords = get_keywords(texts, stop_words=stop)
for i in range(len(extracted_keywords)):
    extracted_keywords[i] = list(filter(match_patterns, extracted_keywords[i]))

for i, etalon in enumerate(keywords):
    tp_count = 0
    used_words = set()
    for word in extracted_keywords[i]:
        for second_word in etalon:
            if second_word not in used_words and (word in second_word or second_word in word):
                used_words.add(second_word)
                tp_count += 1
                break
                
    pr = tp_count/len(extracted_keywords[i])
    rec = tp_count/len(etalon)
    try:
        f_score = 2*pr*rec/(pr + rec)
    except ZeroDivisionError:
        f_score = 0
    print(f"ТЕКСТ {i}")
    print(f"Точность: {pr}")
    print(f"Полнота: {rec}")
    print(f"F-score: {f_score}")
    print("-------------------------------------------")

ТЕКСТ 0
Точность: 0.7142857142857143
Полнота: 0.38461538461538464
F-score: 0.5
-------------------------------------------
ТЕКСТ 1
Точность: 0.25
Полнота: 0.25
F-score: 0.25
-------------------------------------------
ТЕКСТ 2
Точность: 0.0625
Полнота: 0.125
F-score: 0.08333333333333333
-------------------------------------------
ТЕКСТ 3
Точность: 0.17647058823529413
Полнота: 0.375
F-score: 0.24
-------------------------------------------


In [56]:
extracted_keywords[2]

['большой',
 'время',
 'высокий',
 'деятельность',
 'жизнь',
 'который',
 'работа',
 'различный',
 'свой',
 'ситуация',
 'событие',
 'социальный',
 'степень',
 'фактор',
 'человек',
 'эмоциональный']

## Выводы

Морфологические шаблоны изредка повышают точность, но сильно просаживают полноту и как правило мешаются. Разные алгоритмы хорошо работают на разных текстах, но у всех проблемы со вторым и третьим текстом.

Основная проблема как мне кажется - субъективность разметки и степень "спецефичности" каждого слова, например в случае с tf-idf мы выделили много слов, которые, формально говоря специфичны для данного текста, но не были указаны в качестве ключевых так как являются слишком общими (работа, свой, ситуация...). Тут может помочь более жесткий набор стоп-слов, либо просто большее число текстов, чтоб idf-состовляющая стала лучше работать.